# Deezer playlist dataset and song recommendation with word2vec

In this mini project we will develop a word2vec network and use it to build a playlist completion tool (song suggestion). The data is hosted on the following repository: http://github.com/comeetie/deezerplay.git. To know more about word2vec and these data you can read the two following references:

- Efficient estimation of word representations in vector space, Tomas Mikolov, Kai Chen, Greg Corrado, and Jeffrey Dean. (https://arxiv.org/abs/1301.3781)
- Word2with applied to Recommendation: Hyperparameters Matter, H. Caselles-Dupré, F. Lesaint and J. Royo-Letelier. (https://arxiv.org/pdf/1804.04212.pdf)

The elements you have to do are highlighted in red.

## Preparation of data

The data is in the form of a playlist list. Each playlist is a list with the deezer ID of the psong followed by the artist ID.

In [2]:
import numpy as np
data = np.load("data/music_2.npy", allow_pickle=True)
[len(data), np.mean([len(p) for p in data])]

[100000, 24.21338]

The dataset we are going to work on contains 100000 playlists which are composed of an average of 24.1 songs. We will start by keeping only the song identifiers. 

In [3]:
playlist_track = [list(filter(lambda w: w.split("_")[0] == u"track", playlist)) for playlist in data]
playlist_artist = [list(filter(lambda w: w.split("_")[0] == u"artist", playlist)) for playlist in data]

In [4]:
# songs != playlists
tracks = np.unique(np.concatenate(playlist_track))
Vt = len(tracks)
Vt

338509

The number of different songs in this data-set is quite high with more than 300,000 songs.

## Creating a song dictionary
We will assign to each song an integer that will serve as a unique identifier and input for our network. In order to save a little bit of resources we will only work in this project on songs that appear in at least two playlists.

In [5]:
# counting occurences for each track
track_counts = dict((tracks[i], 0) for i in range(0, Vt))
for p in playlist_track:
    for track in p:
        track_counts[track] = track_counts[track] + 1

In [6]:
# Filter very rare songs to save ressources
playlist_track_filter = [list(filter(lambda track : track_counts[track] > 1, playlist)) for playlist in playlist_track]
# get the counts
counts  =  np.array(list(track_counts.values()))
# sort
order = np.argsort(-counts)
# deezed_id array
tracks_list_ordered = np.array(list(track_counts.keys()))[order]
# Vocabulary size = number of kept songs
Vt = np.where(counts[order] == 1)[0][0]
# dict construction id_morceaux num_id [0,Vt]
track_dict = dict((tracks_list_ordered[i], i) for i in range(0, Vt))
# playlist conversion to list of integers
corpus_num_track = [[track_dict[track] for track in play ] for play in playlist_track_filter]


### Creation of test and validation learning sets

To learn the parameters of our method we will keep the first l-1 songs of each playlist (with l the length of the playlist) for learning. To evaluate the completion performance of our method we keep for each playlist the last two songs. The objective will be to find the last one from the next-to-last one. 



In [29]:
# playlist main part used for trainning
play_app  = [corpus_num_track[i][:(len(corpus_num_track[i])-1)] for i in range(len(corpus_num_track)) if len(corpus_num_track[i]) > 1]

# the two last elements are used for validation and training
index_tst = np.random.choice(100000,20000)
index_val = np.setdiff1d(range(100000),index_tst)

play_tst  = np.array([corpus_num_track[i][(len(corpus_num_track[i])-2):len(corpus_num_track[i])] 
             for i in index_tst if len(corpus_num_track[i])>3])
play_val  = np.array([corpus_num_track[i][(len(corpus_num_track[i])-2):len(corpus_num_track[i])] 
             for i in index_val if len(corpus_num_track[i])>3])[:10000]

print(play_val[:,1])


93892
[23294  2016 79201 ...   173 33678   924]


In [8]:
# import Keras
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Flatten
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.preprocessing.sequence import skipgrams, make_sampling_table
import tensorflow as tf

### hyper-parameters of word2vec :

the method word2vec needs some hyper-parameters. We are going to give them the first values, but we will refine them later:


In [9]:
# latent space dimension
# the size of each of our word embedding vectors
vector_dim = 30
# window size
# the window of words around the target word that will be used to draw the context words from
window_width = 3
# number of negative sample per positive sample
neg_sample = 5
# taille des mini-batch
min_batch_size = 50
# smoothing factor for the sampling table of negative pairs 
samp_coef = 0.5
# cparameter to sub-sample frequent song
sub_samp = 0.00001

### Creation of the draw probability tables (smoothed) and unsmoothed

To draw the negative examples we need the smoothed frequencies of each song in our dataset. Likewise to under-sample very frequent pieces we need the raw frequencies. We will calculate these two vectors.

In [10]:
# get the counts
counts = np.array(list(track_counts.values()),dtype='float')[order[:Vt]]
# normalization
st =  counts/np.sum(counts)
# smoothing
st_smooth = np.power(st,samp_coef)
st_smooth = st_smooth/np.sum(st_smooth)

### Building the word2 network with

A word2 network with takes in input two integers corresponding to two songs, these are embedded in a latent space of dimension (vector_dim) thanks to a layer of embedding type (you will have to use the same layer to project the two pieces). Once these two vectors have been extracted, the array must calculate their scalar product normalize appleler cosine distance : 

$$cos(\theta_{ij})=\frac{z_i.z_j}{||z_i||||z_j||}$$

To carry out this treatment you will use a "dot" layer for "dot product". The model then uses a sigmoid layer to produce the output. This output will be 0 when both songs are randomly drawn from the whole dataset and 1 when they were extracted from the same playslist. <span style="color:red">You have to create the keras Track2Vec model corresponding to this architecture.</span>


In [11]:
# inputs

input_target = Input((1,), dtype='int32')
input_context = Input((1,), dtype='int32')

embedding = Embedding(Vt, vector_dim, input_length=1, name='embedding')
target = embedding(input_target)
context = embedding(input_context)
dot_product = Dot(axes=2)([target, context])
flatten = Flatten()(dot_product)
output = Dense(1, activation='sigmoid',name="classif")(flatten)

Track2Vec = Model(inputs=[input_target, input_context], outputs=output)
Track2Vec.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

In [12]:
Track2Vec.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 30)        3697230     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dot (Dot)                       (None, 1, 1)         0           embedding[0][0]              

### Creation of the data generator

To learn the projection layer at the heart of our model we will build a generator of positive and negative pair examples of close or random songs from our training data. The following function will allow us to generate such examples from a playlist (seq) provided as input. This function will first build all the pairs of songs that can be extracted from the playlist if they are within (windows) distance of each other. These pairs will constitute the positive pairs. The pairs concerning very frequent songs will be removed with a probability that depends on their frequencies. Finally a number of negative examples (corresponding to neg_samples * positive number of examples) will be randomly drawn using the neg_sampling_table.

In [13]:
# function to generate word2vec positive and negative pairs 
# from an array of int that represent a text ot here a playlist
# params 
# seq : input text or playlist (array of int)
# neg_samples : number of negative sample to generate per positive ones
# neg_sampling_table : sampling table for negative samples
# sub sampling_table : sampling table for sub sampling common words songs
# sub_t : sub sampling parameter
def word2vecSampling(seq, window, neg_samples, neg_sampling_table, sub_sampling_table, sub_t):
    # vocab size
    V = len(neg_sampling_table)
    # extract positive pairs 
    positives = skipgrams(sequence=seq, vocabulary_size=V, window_size=window, negative_samples=0) #return couples, labels: where couples are int pairs and labels are either 0 or 1.
    ppairs    = np.array(positives[0]) #couples
    # sub sampling
    if (ppairs.shape[0]>0):
        f = sub_sampling_table[ppairs[:,0]]
        subprob = ((f-sub_t)/f)-np.sqrt(sub_t/f)
        tokeep = (subprob<np.random.uniform(size=subprob.shape[0])) | (subprob<0)
        ppairs = ppairs[tokeep,:]
    nbneg     = ppairs.shape[0]*neg_samples
    # sample negative pairs
    if (nbneg > 0):
        negex     = np.random.choice(V, nbneg, p=neg_sampling_table)
        negexcontext = np.repeat(ppairs[:,0],neg_samples)
        npairs    = np.transpose(np.stack([negexcontext,negex]))
        pairs     = np.concatenate([ppairs,npairs],axis=0)
        labels    = np.concatenate([np.repeat(1,ppairs.shape[0]),np.repeat(0,nbneg)])
        perm      = np.random.permutation(len(labels))
        res = [pairs[perm,:],labels[perm]]
    else:
        res=[[],[]]
    return res

<span style="color:red">Use this function to build a "track_ns_generator" of data which will generate positive and negative examples from "nbm" playlists randomly drawn from the "corpus_num" dataset provided as input. </span>

In [14]:
import random
def track_ns_generator(corpus_num,nbm):
    
    
    while 1: 
        x = np.ndarray((0,2), dtype=np.int32)
        y = np.ndarray((0),dtype=np.int32)
        
        for i in range(nbm):
            randint = random.randint(0, len(corpus_num))
            a,b = word2vecSampling(corpus_num[randint], window_width, neg_sample, st_smooth, st, sub_samp)

            if(len(a) > 0):
                x = np.vstack((x,a))
                y = np.append(y,b)
                
        yield ((x[:,0],x[:,1]), y)
        #yield (x, y)
        

## Learning 
You should now be able to learn your first model with the following code. This should take between 15 and 30 min.

In [14]:
hist=Track2Vec.fit(track_ns_generator(play_app,min_batch_size),steps_per_epoch = 200,epochs=60)

Epoch 1/60
200/200 [==============================] - 26s 127ms/step - loss: 0.6771 - accuracy: 0.8140
Epoch 2/60
200/200 [==============================] - 33s 164ms/step - loss: 0.6203 - accuracy: 0.8333
Epoch 3/60
200/200 [==============================] - 33s 163ms/step - loss: 0.5744 - accuracy: 0.8334
Epoch 4/60
200/200 [==============================] - 33s 164ms/step - loss: 0.5337 - accuracy: 0.8344
Epoch 5/60
200/200 [==============================] - 32s 163ms/step - loss: 0.4911 - accuracy: 0.8403
Epoch 6/60
200/200 [==============================] - 33s 164ms/step - loss: 0.4414 - accuracy: 0.8590
Epoch 7/60
200/200 [==============================] - 33s 164ms/step - loss: 0.3884 - accuracy: 0.8842
Epoch 8/60
200/200 [==============================] - 32s 159ms/step - loss: 0.3386 - accuracy: 0.9262
Epoch 9/60
200/200 [==============================] - 32s 162ms/step - loss: 0.2911 - accuracy: 0.9637
Epoch 10/60
200/200 [==============================] - 32s 161ms/step - l

## Save latent space
Once the learning is done, we can save the position of the songs in the latent space with the following code:

In [15]:
# récupérations des positions des morceaux dans l'espace de projection
vectors_tracks = Track2Vec.get_weights()[0]

with open('latent_positions.npy', 'wb') as f:
    np.save(f, vectors_tracks)

And latter load it with :

In [16]:
vectors_tracks=np.load("latent_positions.npy")
print(len(vectors_tracks))

123241


## Use in completion and evaluation
We can now use this space to make suggestions. <span style="color:red">Build a predict_batch function that takes as input a number vector of songs (seeds), (s) a number of suggestions to make per request, the vectors of the songs in the latent space X and a kd-tree to speed up the computation of closest neighbors. To make its propositions this function will return the indices of the s closest neighbors of each seed. </span> So that these predictions don't take too much time you will use a kd-tree (available in scikit learn) to speed up the search for nearest neighbors.

In [17]:
from sklearn.neighbors import KDTree
kdt = KDTree(vectors_tracks, leaf_size=10, metric='euclidean')
print(len(kdt.data))
print(play_val[:,0])

123241
[79029   532 50537 ...  4161  8281  1696]


In [18]:
def predict_batch(seeds, k, X, kdt):
    ind = kdt.query(X, k, return_distance = False)
    return ind[seeds]

<span style="color:red">Use this function to propose songs to complete the playlist of the validation dataset (the seeds correspond to the first column of play_val).</span>

In [19]:
indexes = predict_batch(play_val[:,0],10,vectors_tracks,kdt)
print(indexes)

[[ 79029  30045  87944 ...  14138  63323  47837]
 [   532   2995  78182 ...   1039  28655  26763]
 [ 50537  75167  35510 ...  13606  23544  77281]
 ...
 [ 18195  12075  43924 ...   7709  43634  33916]
 [  9609  17388   2344 ... 118516  88184  93265]
 [     8  98125    206 ...  35305  53617    893]]


<span style="color:red">Compare these suggestions with the second column of play_val (the songs actually present). To do this you will calculate the hit@10 which is 1 if the song actually present in the playlist is one of the 10 suggestions (this score is averaged over the validation set) and the NDCG@10 (Normalized Discounted Cumulative Gain) which takes into account the order of the suggestions. This second score is worth $1/log2(k+1)$ if proposal k (k between 1 and 10) is the correct proposal and 0 if no proposal is correct. As before you will calculate the average score on the validation set. </span>


In [19]:
import math
def NDGCatK(indexes):
    NDGCatK = 0
    for i in range(len(play_val[:,1])):
        for j in range(len(indexes[i])):
            if play_val[i,1] == indexes[i][j]:
                NDGCatK += 1/(math.log(j+1,2))

    NDGCatK = NDGCatK/len(play_val[:,1])
    print(NDGCatK)
    
    
NDGCatK(indexes)

NameError: name 'indexes' is not defined

In [21]:
def HitatK(indexes):
    HitatK = 0
    for i in range(len(indexes)):
        if play_val[i,1] in indexes[i]:
            HitatK += 1
    HitatK = HitatK/len(play_val[:,1])
    print(HitatK)
    
HitatK(indexes)

0.0925

## hyper parameters tunning

<span style="color:red">You can now try to vary the hyper parameters to improve your performance. Pay attention to the computing time : prepare a grid with about ten different configurations and evaluate each of them on your validation set.
Evaluate the final performance of the best configuration found on the test set. Don't forget to save your results.</span>



Possiamo cambiare: epochs (32,...), steps_per_epoch, batch_size, optimizer, forse anche la loss function

In [28]:
#1
min_batch_size = 32
hist1=Track2Vec.fit(track_ns_generator(play_app,min_batch_size),steps_per_epoch = 400,epochs=50)

kdt = KDTree(vectors_tracks, leaf_size=10, metric='euclidean')
indexes = predict_batch(play_val[:,0],10,vectors_tracks,kdt)
NDGCatK(indexes)
HitatK(indexes)



#2
min_batch_size = 64
Track2Vec.compile(loss='binary_crossentropy', optimizer='SGD', metrics=["accuracy"])
hist2=Track2Vec.fit(track_ns_generator(play_app,min_batch_size),steps_per_epoch = 350,epochs=32)

kdt = KDTree(vectors_tracks, leaf_size=10, metric='euclidean')
indexes = predict_batch(play_val[:,0],10,vectors_tracks,kdt)
NDGCatK(indexes)
HitatK(indexes)



#3
min_batch_size = 64
Track2Vec.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
hist3=Track2Vec.fit(track_ns_generator(play_app,min_batch_size),steps_per_epoch = 350,epochs=32)

kdt = KDTree(vectors_tracks, leaf_size=10, metric='euclidean')
indexes = predict_batch(play_val[:,0],10,vectors_tracks,kdt)
NDGCatK(indexes)
HitatK(indexes)



#4
min_batch_size = 100
Track2Vec.compile(loss='hinge', optimizer='adam', metrics=["accuracy"])
hist4=Track2Vec.fit(track_ns_generator(play_app,min_batch_size),steps_per_epoch = 200,epochs=50)

kdt = KDTree(vectors_tracks, leaf_size=10, metric='euclidean')
indexes = predict_batch(play_val[:,0],10,vectors_tracks,kdt)
NDGCatK(indexes)
HitatK(indexes)



#5
min_batch_size = 100
Track2Vec.compile(loss='hinge', optimizer='adam', metrics=["accuracy"])
hist5=Track2Vec.fit(track_ns_generator(play_app,min_batch_size),steps_per_epoch = 200,epochs=50)

kdt = KDTree(vectors_tracks, leaf_size=10, metric='euclidean')
indexes = predict_batch(play_val[:,0],10,vectors_tracks,kdt)
NDGCatK(indexes)
HitatK(indexes)


#6
min_batch_size = 64
Track2Vec.compile(loss='squared_hinge', optimizer='adam', metrics=["accuracy"])
hist5=Track2Vec.fit(track_ns_generator(play_app,min_batch_size),steps_per_epoch = 200,epochs=32)

kdt = KDTree(vectors_tracks, leaf_size=10, metric='euclidean')
indexes = predict_batch(play_val[:,0],10,vectors_tracks,kdt)
NDGCatK(indexes)
HitatK(indexes)



TypeError: Singleton array array(<generator object track_ns_generator at 0x000001EF43C61F48>,
      dtype=object) cannot be considered a valid collection.

## Bonus, a little music

The TrackArtists file contains meta.data on the tracks and the artists for a subset of the 300,000 tracks in the dataset. We can use it to search for the number of a song from its title:

In [ ]:
import pandas as pd
tr_meta=pd.read_csv("./TracksArtists.csv")
joindf = pd.DataFrame({"track_id":tracks_list_ordered[:Vt],"index":range(Vt)})
meta = tr_meta.merge(joindf, left_on="track_id",right_on="track_id")
meta.set_index("index",inplace=True)
meta[["title","name","preview","track_id"]]

In [ ]:
def find_track(title):
    return meta.loc[meta["title"]==title,:].index[0]

tr=find_track("Hexagone")
tr

## Radio

The deeezer api allows you to retrieve information about the pieces of the dataset from their deezer id. Among this information when it is available a url to listen to a free sample is provided.

In [ ]:
import urllib.request, json 
def gettrackinfo(number):
    track_url =  "https://api.deezer.com/track/{}".format(tracks_list_ordered[number].split("_")[1])
    with urllib.request.urlopen(track_url) as url:
        data = json.loads(url.read().decode())
    return data
track_apidata = gettrackinfo(find_track("Hexagone"))
track_apidata

So we can use it to listen a preview:

In [ ]:
from IPython.display import display, Audio, clear_output
display(Audio(track_apidata["preview"],autoplay=True))

<span style="color:red">Create a radio function that takes as input a track number in the dataset and launches a series of nb_track tracks by randomly pulling in the neighborhood of the current track the next track to listen to. The size of the neighborhood will be configurable and you will delete from the proposals the songs already listened to. You will handle exceptions if the track does not have an available extract. You can delete the current song with the clear_display function.</span>

In [ ]:
import time
def start_radio(seed,nb_candidates,duration,nbsteps=20):
    print(meta.loc[seed,"title"])
    display(Audio(meta.loc[seed,"preview"],autoplay=True))
    time.sleep(duration)
    clear_output()
    already_played = [seed]
    for i in range(nbsteps):
        try:
            # TODO
        except:
            print("track not found")
            pass
        clear_output()

In [ ]:
start_radio(find_track("Hexagone"),5,5,10)